## Notebook for making predictions with deleted constant y samples


In [1]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [2]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [9]:
X_A_train = dm.data_A.iloc[:, 2:-1]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:-1]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:-1]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

Y_A_train


0          0.00
1          0.00
2          0.00
3          0.00
4         19.36
          ...  
34054    692.56
34055    194.70
34056      9.02
34057      0.00
34060      0.00
Name: pv_measurement, Length: 34017, dtype: float64

In [14]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

In [20]:
X_B_train.shape

(104574, 25)

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [10]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

In [11]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

Learning rate set to 0.07148
0:	learn: 1100.5745334	total: 159ms	remaining: 2m 38s
1:	learn: 1040.6873648	total: 165ms	remaining: 1m 22s
2:	learn: 985.5618557	total: 171ms	remaining: 56.8s
3:	learn: 934.5507003	total: 176ms	remaining: 43.9s
4:	learn: 888.1712394	total: 182ms	remaining: 36.1s
5:	learn: 845.0604252	total: 188ms	remaining: 31.1s
6:	learn: 806.6655029	total: 193ms	remaining: 27.3s
7:	learn: 770.7584278	total: 198ms	remaining: 24.6s
8:	learn: 738.5101255	total: 205ms	remaining: 22.5s
9:	learn: 709.4172249	total: 211ms	remaining: 20.9s
10:	learn: 682.5056167	total: 217ms	remaining: 19.5s
11:	learn: 658.0359170	total: 223ms	remaining: 18.4s
12:	learn: 636.4216506	total: 229ms	remaining: 17.4s
13:	learn: 616.5458972	total: 234ms	remaining: 16.4s
14:	learn: 598.5732088	total: 239ms	remaining: 15.7s
15:	learn: 582.5216645	total: 244ms	remaining: 15s
16:	learn: 567.8433433	total: 250ms	remaining: 14.4s
17:	learn: 554.6055620	total: 255ms	remaining: 13.9s
18:	learn: 542.5914497	to

## Preds

In [8]:
pd.set_option('display.max_columns', None)
X_B_test.head(3)

,absolute_humidity_2m:gm3,ceiling_height_agl:m,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,fresh_snow_1h:cm,fresh_snow_24h:cm,is_in_shadow:idx,msl_pressure:hPa,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_melt_10min:mm,snow_water:kgm2,sun_elevation:d,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_w_1000hPa:ms,date_forecast,month,season,hours,sum_rad:W,rad_cloud_ratio
0,4.30,912.299988,1059.750000,0.0,271.650024,0.0,0.0,1.0,1013.775024,0.0,1006.674988,0.0,0.0,80.250000,1012.974976,0.0,0.0,-10.54025,75.000000,31107.599609,3.950,0.0,2023-05-01 00:00:00,0.522008,0.809017,0,0.0,0.0
1,4.25,1875.195190,1073.699951,0.0,271.450012,0.0,0.0,1.0,1013.250000,0.0,1006.174988,0.0,0.0,79.800003,1012.474976,0.0,0.0,-7.89400,77.500000,30409.699219,3.825,0.0,2023-05-01 01:00:00,0.522008,0.809017,1,0.0,0.0
2,4.15,1765.900024,1200.099976,0.0,271.049988,0.0,0.0,1.0,1012.700012,0.0,1005.650024,0.0,0.0,78.000000,1011.950012,0.0,0.0,-3.81500,88.099998,31342.650391,3.650,0.0,2023-05-01 02:00:00,0.522008,0.809017,2,0.0,0.0


In [12]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

In [13]:
df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("log_sub_4.csv")

df.shape

(2160, 1)